<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/disneyland_crowdLevel_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? 
Your response ('') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 3.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.9 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=10ec0f35b3114d6fde4bc74059bbb7306a5d4244865455047c4058af03ab9077
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 18 19:39 kaggle.json


In [3]:
!kaggle competitions download -c disneyland-crowd-levels

  0% 0.00/4.15k [00:00<?, ?B/s]
100% 4.15k/4.15k [00:00<00:00, 3.87MB/s]


In [4]:
!unzip disneyland-crowd-levels.zip

Archive:  disneyland-crowd-levels.zip
  inflating: submission_sample.csv   
  inflating: test.csv                
  inflating: train.csv               


In [306]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [307]:
xy = pd.read_csv('train.csv',header=None)
scaler = MinMaxScaler()
x_data = xy.loc[1:,1:4]
y_data = xy.loc[1:, 7:]
x_data=np.array(x_data, dtype=int)
y_data=np.array(y_data, dtype=int)
mintemp=x_data[:,0].min()
x_data[:,0]=x_data[:,0]-mintemp
scaler.fit(x_data)
x_data=scaler.transform(x_data)
x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)
nb_class = 4
nb_data = len(y_train)
linear1 = torch.nn.Linear(4,100,bias=True)
linear3 = torch.nn.Linear(100,4,bias=True)
relu = torch.nn.ReLU()
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

dropout = torch.nn.Dropout(p=0.5)
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear3)

In [308]:
y_train=y_train.reshape([761])
y_train=y_train-1

In [309]:

loss = torch.nn.CrossEntropyLoss()
op=optim.Adam(model.parameters(),lr=1e-3)
epochs= 15000
minloss=1000
model=model.cuda()
loss=loss.cuda()
for epoch in range(epochs) : 
    rand= np.random.choice(np.array([i for i in range(761)]),761) 
    avg=[]
    op.zero_grad()
    output =model(x_train.cuda())
    cost=loss(output,y_train.cuda())
    cost.backward()
    op.step()
    avg.append(cost.item())
    if np.array(avg).mean()<minloss:
      minloss=np.array(avg).mean()
      goodmodel=model
    if(epoch%100==0):
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, minloss))

Epoch    0/15000 Cost: 1.414204
Epoch  100/15000 Cost: 1.180354
Epoch  200/15000 Cost: 1.152237
Epoch  300/15000 Cost: 1.139431
Epoch  400/15000 Cost: 1.134808
Epoch  500/15000 Cost: 1.132558
Epoch  600/15000 Cost: 1.125065
Epoch  700/15000 Cost: 1.122652
Epoch  800/15000 Cost: 1.122652
Epoch  900/15000 Cost: 1.120414
Epoch 1000/15000 Cost: 1.120030
Epoch 1100/15000 Cost: 1.115243
Epoch 1200/15000 Cost: 1.113188
Epoch 1300/15000 Cost: 1.111875
Epoch 1400/15000 Cost: 1.110736
Epoch 1500/15000 Cost: 1.110736
Epoch 1600/15000 Cost: 1.107335
Epoch 1700/15000 Cost: 1.104535
Epoch 1800/15000 Cost: 1.104535
Epoch 1900/15000 Cost: 1.104535
Epoch 2000/15000 Cost: 1.101299
Epoch 2100/15000 Cost: 1.101299
Epoch 2200/15000 Cost: 1.099288
Epoch 2300/15000 Cost: 1.099288
Epoch 2400/15000 Cost: 1.098388
Epoch 2500/15000 Cost: 1.098388
Epoch 2600/15000 Cost: 1.097194
Epoch 2700/15000 Cost: 1.097194
Epoch 2800/15000 Cost: 1.097194
Epoch 2900/15000 Cost: 1.097194
Epoch 3000/15000 Cost: 1.097194
Epoch 31

In [310]:
testxy = pd.read_csv('test.csv', header=None)
tx_data = testxy.loc[1:,1:4]
tx_data=np.array(tx_data, dtype=int)
tx_data[:,0]=tx_data[:,0]-mintemp
tx_data=scaler.transform(tx_data)
x_test = torch.FloatTensor(tx_data)

In [311]:
with torch.no_grad():
  output=goodmodel(x_test.cuda())
  prediction = torch.argmax(output, dim=1)
  prediction=prediction+1


In [312]:
prediction

tensor([2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 3, 3, 3, 3, 2, 2, 3, 2, 3, 3, 3, 3, 3,
        2, 2, 2, 2, 2], device='cuda:0')

In [313]:
submission=pd.read_csv("submission_sample.csv")
for i in range(len(prediction)):
    submission['Crowd level'][i] = prediction[i].item()
submission['Crowd level']=submission['Crowd level'].astype("int")
submission.to_csv("out.csv",index=False,header=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
